## This script is to scrape public websites to get published population details of countries part of the analysis 

In [1]:
import pandas as pd
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
def scrape_url(url):
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    type(soup)
    title = soup.title
    text = soup.get_text()
    rows = soup.find_all('tr')
    list_rows = []
    for row in rows:
        cells = row.find_all('td')
        str_cells = str(cells)
        clean2 = (re.sub(r'[^\x00-\x7f]', '', (re.sub(r'\[.\]', '', (re.sub(r'(?<=\d),(?=\d)', '', (re.sub('\n', '', (re.sub(r'<.*?>', '', str_cells))))))))))
        list_rows.append(clean2)
        type(clean2)
    df = pd.DataFrame(list_rows)
    df1 = df[0].str.split(',', expand=True)
    return df1

In [3]:
#scrape 1st URL
url = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"
df1 = scrape_url(url)
df1.columns = [ 'rank', 'country', 'population', 'date1', 'date2', '%population', 'source','temp']
df1['rank'] = df1['rank'].str.replace('[','')
df1['country'] = df1['country'].str.replace(r"\[.*\]","").str.replace(' ', "")
df1['date'] = df1['date1'] +  df1['date2']
df1 = df1.iloc[1:-13,]
df1 = df1.set_index('rank')
df1 = df1.drop(['temp', 'date1', 'date2', 'source'], axis=1)
df1.to_csv('./countries_population.csv', header=True, sep=',')
report = pd.read_csv('./model_summary.csv')
merge1 = pd.merge(report,df1, how='left', on = 'country')

#scrape 2nd URL
url = "https://en.wikipedia.org/wiki/List_of_countries_by_population_growth_rate"
df2 = scrape_url(url)
df2 = df2.iloc[244:483,]
cols = [x for x in range(7,1922)]
df2 = df2.drop(df2.columns[cols],axis=1)
df2.columns =['country', '2009', '2012', '2014', '2005-2010', '2010-2015', '2015-2020']
df2['country'] = df2['country'].str.replace(r"\[.*\]","").str.replace(' ', "").str.replace('[','')
df2['2015-2020'] = df2['2015-2020'].str.replace(' ', "").str.replace(']', '')
df2 = df2.set_index('country')
df2.to_csv('./countries_population_change.csv', header=True, sep=',')
summary = pd.merge(merge1, df2, how='left', on = 'country')
summary.to_csv('./report_summary.csv', header=True, index=False, sep=',')